<a href="https://colab.research.google.com/github/arbouria/Notas-Aprendizaje-y-Comportamiento-Adaptable-I/blob/main/doc/notebooks/interactive_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PyDDM online demo

This is an interactive demo of a GDDM with leaky integration and exponentially collapsing bounds.

This demo can be run like a normal Jupyter notebook.  If you've never used Jupyter notebooks before, hover over both headings below ("Install PyDDM on Google Colab" and "Define the model and run the GUI") and press the play button on each.  An interactive demo will show below.  To make changes to the model and try out your changes, click on "Show code" and edit it.  When you are done, click on the play button again to update the demo.

In [ ]:
#@title Install PyDDM on Google Colab
!pip -q install git+https://github.com/mwshinn/PyDDM

In [ ]:
#@title Define the model and run the GUI
import pyddm
import pyddm.plot
import numpy as np
model = pyddm.gddm(drift=lambda x,leak,driftrate : driftrate - x*leak,
                   noise=1,
                   bound=lambda t,initial_B,collapse_rate : initial_B * np.exp(-collapse_rate*t),
                   starting_position="x0",
                   parameters={"leak": (0, 2),
                               "driftrate": (-3, 3),
                               "initial_B": (.5, 1.5),
                               "collapse_rate": (0, 10),
                               "x0": (-.9, .9)})

pyddm.plot.model_gui_jupyter(model)
# pyddm.plot.model_gui(model) # If not using a Jupyter notebook or Google Colab

# Demostración Paso a Paso: Convolución 2D

Este código ilustra cómo funciona la operación de convolución en 2D, que es fundamental en las CNNs. Se desliza un filtro (kernel


In [7]:
# Celda para Convolución 2D Paso a Paso

import numpy as np

def convolve2d_step_by_step(input_matrix, kernel, stride=1, padding=0):
    """Realiza convolución 2D y muestra el primer paso."""

    # Añadir padding si es necesario (aquí no lo haremos para simplificar)
    if padding > 0:
      input_matrix = np.pad(input_matrix, pad_width=padding, mode='constant', constant_values=0)

    # Dimensiones
    input_h, input_w = input_matrix.shape
    kernel_h, kernel_w = kernel.shape

    # Calcular dimensiones de salida
    output_h = int(((input_h - kernel_h) / stride) + 1)
    output_w = int(((input_w - kernel_w) / stride) + 1)

    # Crear matriz de salida (mapa de características) inicializada con ceros
    output_matrix = np.zeros((output_h, output_w))

    print("--- Convolución 2D Paso a Paso ---")
    print("Matriz de Entrada (Imagen):")
    print(input_matrix)
    print("\nFiltro (Kernel):")
    print(kernel)
    print(f"\nStride: {stride}, Padding: {padding}")
    print(f"Tamaño de Salida Esperado: ({output_h} x {output_w})")
    print("-" * 30)

    # --- Mostrar el cálculo para el PRIMER valor de salida (output[0, 0]) ---
    print("Calculando el primer valor del mapa de características (output[0, 0]):")

    # 1. Extraer la región de entrada correspondiente
    start_row, start_col = 0, 0 # Para output[0,0], empezamos en input[0,0]
    end_row = start_row + kernel_h
    end_col = start_col + kernel_w
    input_patch = input_matrix[start_row:end_row, start_col:end_col]
    print("\n1. Región de la entrada cubierta por el filtro (Input Patch):")
    print(input_patch)

    # 2. Multiplicación elemento por elemento
    elementwise_product = input_patch * kernel
    print("\n2. Multiplicación Elemento por Elemento (Input Patch * Kernel):")
    print(elementwise_product)

    # 3. Suma de los resultados
    output_value = np.sum(elementwise_product)
    print(f"\n3. Suma de los productos = {output_value}")
    output_matrix[0, 0] = output_value
    print("\n--> Este valor (", output_value, ") va en la posición (0, 0) del mapa de características.")
    print("-" * 30)


    # --- Calcular el resto del mapa de características (sin mostrar cada paso) ---
    print("Calculando el resto del mapa de características...")
    for r_out in range(output_h):
        for c_out in range(output_w):
            # No recalculamos el [0,0] que ya hicimos
            if r_out == 0 and c_out == 0:
                continue

            # Encontrar esquinas en la entrada
            start_row = r_out * stride
            start_col = c_out * stride
            end_row = start_row + kernel_h
            end_col = start_col + kernel_w

            # Extraer parche
            input_patch = input_matrix[start_row:end_row, start_col:end_col]

            # Calcular convolución para esta posición
            output_matrix[r_out, c_out] = np.sum(input_patch * kernel)

    print("\nMapa de Características Final (Salida de la Convolución):")
    print(output_matrix)
    print("-" * 30)
    return output_matrix


# --- Ejemplo de Uso ---
# Matriz de entrada (ejemplo 5x5)
input_data = np.array([
    [5, 3, 1, 0, 2],
    [8, 7, 5, 3, 1],
    [2, 0, 1, 5, 3],
    [4, 1, 3, 2, 0],
    [1, 8, 5, 2, 1]
])

# Filtro (ejemplo 3x3 - podría ser un detector de bordes simple o de promedio)
# kernel_filter = np.array([
#     [-1, -1, -1],
#     [-1,  8, -1],
#     [-1, -1, -1]
# ]) # Detector de bordes

kernel_filter = np.array([
    [0.1, 0.2, 0.1],
    [0.3, 0.5, 0.3],
    [0.1, 0.2, 0.1]
]) # Filtro de promedio ponderado (como en la diapositiva 24)


# Ejecutar la función
feature_map = convolve2d_step_by_step(input_data, kernel_filter, stride=1, padding=0)


--- Convolución 2D Paso a Paso ---
Matriz de Entrada (Imagen):
[[5 3 1 0 2]
 [8 7 5 3 1]
 [2 0 1 5 3]
 [4 1 3 2 0]
 [1 8 5 2 1]]

Filtro (Kernel):
[[0.1 0.2 0.1]
 [0.3 0.5 0.3]
 [0.1 0.2 0.1]]

Stride: 1, Padding: 0
Tamaño de Salida Esperado: (3 x 3)
------------------------------
Calculando el primer valor del mapa de características (output[0, 0]):

1. Región de la entrada cubierta por el filtro (Input Patch):
[[5 3 1]
 [8 7 5]
 [2 0 1]]

2. Multiplicación Elemento por Elemento (Input Patch * Kernel):
[[0.5 0.6 0.1]
 [2.4 3.5 1.5]
 [0.2 0.  0.1]]

3. Suma de los productos = 8.9

--> Este valor ( 8.9 ) va en la posición (0, 0) del mapa de características.
------------------------------
Calculando el resto del mapa de características...

Mapa de Características Final (Salida de la Convolución):
[[8.9 6.7 5. ]
 [4.5 4.9 5.6]
 [5.1 5.1 4.3]]
------------------------------


# Demostración Paso a Paso: Max Pooling

Este código ilustra la operación de Max Pooling, comúnmente usada después de la convolución y activación en CNNs. Reduce la dimensionalidad del mapa de características y proporciona invarianza a pequeñas traslaciones, seleccionando el valor máximo en cada ventana de pooling. Mostraremos el cálculo para el primer valor del mapa agrupado (pooled map).

In [8]:
# Celda para Max Pooling 2D Paso a Paso

import numpy as np

def max_pool_step_by_step(feature_map, pool_size=2, stride=2):
    """Realiza Max Pooling 2D y muestra el primer paso."""

    # Dimensiones
    map_h, map_w = feature_map.shape

    # Calcular dimensiones de salida
    output_h = int(((map_h - pool_size) / stride) + 1)
    output_w = int(((map_w - pool_size) / stride) + 1)

    # Crear matriz de salida (pooled map) inicializada con ceros
    pooled_map = np.zeros((output_h, output_w))

    print("\n--- Max Pooling 2D Paso a Paso ---")
    print("Mapa de Características de Entrada:")
    print(feature_map)
    print(f"\nTamaño de Ventana (Pool Size): {pool_size}x{pool_size}, Stride: {stride}")
    print(f"Tamaño de Salida Esperado: ({output_h} x {output_w})")
    print("-" * 30)

    # --- Mostrar el cálculo para el PRIMER valor de salida (pooled_map[0, 0]) ---
    print("Calculando el primer valor del mapa agrupado (pooled_map[0, 0]):")

    # 1. Extraer la región de entrada correspondiente
    start_row, start_col = 0, 0 # Para output[0,0], empezamos en input[0,0]
    end_row = start_row + pool_size
    end_col = start_col + pool_size
    input_patch = feature_map[start_row:end_row, start_col:end_col]
    print("\n1. Región del mapa cubierta por la ventana de pooling:")
    print(input_patch)

    # 2. Encontrar el valor máximo en la región
    max_value = np.max(input_patch)
    print(f"\n2. Valor Máximo en la región = {max_value}")
    pooled_map[0, 0] = max_value
    print("\n--> Este valor (", max_value, ") va en la posición (0, 0) del mapa agrupado.")
    print("-" * 30)

    # --- Calcular el resto del mapa agrupado (sin mostrar cada paso) ---
    print("Calculando el resto del mapa agrupado...")
    for r_out in range(output_h):
        for c_out in range(output_w):
             # No recalculamos el [0,0] que ya hicimos
            if r_out == 0 and c_out == 0:
                continue

            # Encontrar esquinas en la entrada
            start_row = r_out * stride
            start_col = c_out * stride
            end_row = start_row + pool_size
            end_col = start_col + pool_size

            # Extraer parche
            input_patch = feature_map[start_row:end_row, start_col:end_col]

            # Calcular max pooling para esta posición
            pooled_map[r_out, c_out] = np.max(input_patch)

    print("\nMapa Agrupado Final (Salida del Pooling):")
    print(pooled_map)
    print("-" * 30)
    return pooled_map


# --- Ejemplo de Uso ---
# Usaremos un mapa de características de ejemplo (podría ser la salida
# de la convolución anterior, o uno nuevo más grande para ver mejor el pooling)

# Opción 1: Usar la salida de la convolución anterior (si fue 3x3, el pooling 2x2 no cabe bien con stride 2)
# print("\nUsando la salida de la convolución anterior como entrada para Pooling:")
# Si feature_map tiene el tamaño correcto (ej. 4x4 o más), podemos usarlo:
# pooled_output = max_pool_step_by_step(feature_map, pool_size=2, stride=2)

# Opción 2: Definir un nuevo mapa de características para el ejemplo de pooling (ej. 4x4)
print("\nDefiniendo un nuevo mapa de características para el ejemplo de Pooling:")
feature_map_pooling = np.array([
    [8.9, 6.5, 4.2, 3.1],
    [5.1, 9.3, 7.0, 2.5],
    [2.8, 1.2, 5.7, 8.4],
    [4.0, 3.3, 1.9, 6.6]
])

# Ejecutar la función de pooling
pooled_output = max_pool_step_by_step(feature_map_pooling, pool_size=2, stride=2)

# Ejemplo como el de la diapositiva 28
print("\nEjemplo similar al de la diapositiva 28:")
slide_example_map = np.array([
    [1, 1, 2, 4],
    [5, 6, 7, 8],
    [3, 2, 1, 0],
    [1, 2, 3, 4]
])
pooled_slide = max_pool_step_by_step(slide_example_map, pool_size=2, stride=2)





Definiendo un nuevo mapa de características para el ejemplo de Pooling:

--- Max Pooling 2D Paso a Paso ---
Mapa de Características de Entrada:
[[8.9 6.5 4.2 3.1]
 [5.1 9.3 7.  2.5]
 [2.8 1.2 5.7 8.4]
 [4.  3.3 1.9 6.6]]

Tamaño de Ventana (Pool Size): 2x2, Stride: 2
Tamaño de Salida Esperado: (2 x 2)
------------------------------
Calculando el primer valor del mapa agrupado (pooled_map[0, 0]):

1. Región del mapa cubierta por la ventana de pooling:
[[8.9 6.5]
 [5.1 9.3]]

2. Valor Máximo en la región = 9.3

--> Este valor ( 9.3 ) va en la posición (0, 0) del mapa agrupado.
------------------------------
Calculando el resto del mapa agrupado...

Mapa Agrupado Final (Salida del Pooling):
[[9.3 7. ]
 [4.  8.4]]
------------------------------

Ejemplo similar al de la diapositiva 28:

--- Max Pooling 2D Paso a Paso ---
Mapa de Características de Entrada:
[[1 1 2 4]
 [5 6 7 8]
 [3 2 1 0]
 [1 2 3 4]]

Tamaño de Ventana (Pool Size): 2x2, Stride: 2
Tamaño de Salida Esperado: (2 x 2)
-------